In [ ]:
!pip install requests beautifulsoup4 sentence-transformers scikit-learn plotly pandas konlpy
!pip install --upgrade nbformat

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
import re
from konlpy.tag import Okt # 한국어 형태소 분석기

# --- Java 환경 변수 설정 ---
# 이 경로를 위에서 찾은 실제 JDK 경로로 변경해주세요!
# 예시: "/usr/lib/jvm/java-11-openjdk-amd64"
java_home_path = "/usr/lib/jvm/java-11-openjdk-amd64" # **여기를 본인의 실제 JDK 경로로 수정하세요.**

os.environ["JAVA_HOME"] = java_home_path
print(f"JAVA_HOME이 '{os.environ['JAVA_HOME']}'으로 설정되었습니다.")

try:
    okt = Okt() # Okt 객체는 단 한 번만 생성합니다.
    print("Okt 객체가 성공적으로 생성되었습니다!")
except Exception as e:
    print(f"Okt 객체 생성 중 오류 발생: {e}")
    print("JAVA_HOME 경로를 다시 확인하거나, JDK가 제대로 설치되었는지 확인해주세요.")
    # Okt 객체 생성이 실패하면 이후 코드를 실행해도 의미가 없으므로, 필요에 따라 sys.exit() 등을 사용할 수 있습니다.

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


JAVA_HOME이 '/usr/lib/jvm/java-11-openjdk-amd64'으로 설정되었습니다.
Okt 객체가 성공적으로 생성되었습니다!


In [3]:
import os
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
import re
from konlpy.tag import Okt # 한국어 형태소 분석기

# --- Java 환경 변수 설정 ---
# 이 경로를 위에서 찾은 실제 JDK 경로로 변경해주세요!
# 예시: "/usr/lib/jvm/java-11-openjdk-amd64"
java_home_path = "/usr/lib/jvm/java-11-openjdk-amd64" # **여기를 본인의 실제 JDK 경로로 수정하세요.**

os.environ["JAVA_HOME"] = java_home_path
print(f"JAVA_HOME이 '{os.environ['JAVA_HOME']}'으로 설정되었습니다.")

try:
    okt = Okt() # Okt 객체는 단 한 번만 생성합니다.
    print("Okt 객체가 성공적으로 생성되었습니다!")
except Exception as e:
    print(f"Okt 객체 생성 중 오류 발생: {e}")
    print("JAVA_HOME 경로를 다시 확인하거나, JDK가 제대로 설치되었는지 확인해주세요.")
    # Okt 객체 생성이 실패하면 이후 코드를 실행해도 의미가 없으므로, 필요에 따라 sys.exit() 등을 사용할 수 있습니다.

JAVA_HOME이 '/usr/lib/jvm/java-11-openjdk-amd64'으로 설정되었습니다.
Okt 객체가 성공적으로 생성되었습니다!


In [4]:
# 웹 페이지 텍스트 추출 함수
def get_text_from_url(url):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status() # HTTP 오류 발생 시 예외 발생
        soup = BeautifulSoup(response.text, 'html.parser')

        # 스크립트, 스타일 태그 제거
        for script_or_style in soup(['script', 'style']):
            script_or_style.extract()

        # 텍스트만 추출
        text = soup.get_text()

        # 여러 공백을 하나의 공백으로, 줄바꿈 제거
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    except requests.exceptions.RequestException as e:
        print(f"URL에서 데이터를 가져오는 데 실패했습니다: {e}")
        return None
    except Exception as e:
        print(f"텍스트 처리 중 오류 발생: {e}")
        return None

# 한국어 명사(키워드) 추출 함수 (빈도 반환 포함)
def extract_korean_nouns(text, min_length=2, top_n=100, stopwords=None):
    if stopwords is None:
        stopwords = []

    nouns = okt.nouns(text)
    filtered_nouns = [
        word for word in nouns
        if len(word) >= min_length and
           not re.match(r'^[0-9]+$', word) and # 숫자만 있는 단어 제거
           not re.match(r'^[^\w\s]+$', word) and # 특수문자만 있는 단어 제거
           word not in stopwords # 불용어 목록에 없는 단어만 포함
    ]
    
    word_counts = pd.Series(filtered_nouns).value_counts()
    
    # 상위 N개 단어만 선택
    top_words_series = word_counts.head(top_n)
    
    # 단어 리스트와 해당 빈도 리스트를 별도로 반환합니다.
    top_words = top_words_series.index.tolist()
    top_frequencies = top_words_series.values.tolist()
    
    return top_words, top_frequencies

# 단어 임베딩 모델 로드 (다국어 지원 모델 사용)
model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

# 단어 임베딩 함수
def embed_words(words, model):
    if not words:
        return None
    print(f"{len(words)}개의 단어를 임베딩 중...")
    embeddings = model.encode(words, show_progress_bar=True)
    return embeddings

# 3D 차원 축소 함수 (n_components=3으로 고정)
def reduce_dimensions_3d(embeddings):
    if embeddings is None or len(embeddings) < 3:
        print(f"경고: 임베딩 개수가 3보다 적어 3D 차원 축소를 수행할 수 없습니다.")
        return None
    
    # perplexity 값을 데이터 개수에 맞게 조정 (TSNE 오류 방지)
    # TSNE requires perplexity > 1 and perplexity < n_samples
    perplexity_val = min(30, len(embeddings) - 1)
    if perplexity_val <= 1:
        print("경고: TSNE를 위한 충분한 데이터 포인트가 없습니다 (perplexity <= 1).")
        return None
        
    tsne = TSNE(n_components=3, random_state=42, perplexity=perplexity_val)
    reduced_embeddings = tsne.fit_transform(embeddings)
    return reduced_embeddings

In [5]:
# 시각화할 웹 페이지 URL 입력
url_to_visualize = "https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5" # 위키백과 인공지능 페이지 예시
# 다른 예시 URL: "https://www.google.com/intl/ko/about/products/" (텍스트 양이 많지 않을 수 있습니다)
# 원하는 URL로 변경하세요.

print(f"'{url_to_visualize}' 에서 텍스트를 추출하고 있습니다. 잠시만 기다려 주세요...")
text_content = get_text_from_url(url_to_visualize)

if text_content:
    print("\n--- 추출된 텍스트 미리보기 (상위 500자) ---")
    print(text_content[:500] + "..." if len(text_content) > 500 else text_content)

    print("\n--- 키워드 (명사) 추출 중 ---")
    # extract_korean_nouns 함수가 단어와 빈도를 함께 반환하도록 변경했으므로, 두 변수로 받습니다.
    # 사용자 정의 불용어를 추가하고 싶다면 stopwords=korean_stopwords 인자를 전달할 수 있습니다.
    keywords, frequencies = extract_korean_nouns(text_content, top_n=100) 
    
    # 선택 사항: 불용어 목록 예시 (주석 해제 후 사용)
    # korean_stopwords = ['대한', '통해', '경우', '있습니다', '됩니다', '같은', '가지', '등', '수', '것', '때', '이', '그', '저']
    # keywords, frequencies = extract_korean_nouns(text_content, top_n=100, stopwords=korean_stopwords)
    
    print(f"총 {len(keywords)}개의 주요 키워드가 추출되었습니다.")
    print("추출된 키워드 (상위 20개):", keywords[:20])
    print("해당 빈도 (상위 20개):", frequencies[:20])

    if not keywords:
        print("경고: 웹 페이지에서 유효한 키워드를 추출할 수 없었습니다.")
else:
    print("오류: 웹 페이지에서 텍스트를 가져오거나 처리하는 데 문제가 발생했습니다.")

'https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5' 에서 텍스트를 추출하고 있습니다. 잠시만 기다려 주세요...

--- 추출된 텍스트 미리보기 (상위 500자) ---
인공지능 - 위키백과, 우리 모두의 백과사전 본문으로 이동 주 메뉴 주 메뉴 사이드바로 이동 숨기기 둘러보기 대문최근 바뀜요즘 화제임의의 문서로 사용자 모임 사랑방사용자 모임관리 요청 편집 안내 소개도움말정책과 지침질문방 검색 검색 보이기 기부 계정 만들기 로그인 개인 도구 기부 계정 만들기 로그인 목차 사이드바로 이동 숨기기 처음 위치 1 강인공지능과 약인공지능 강인공지능과 약인공지능 하위섹션 토글하기 1.1 약인공지능 1.2 강인공지능 (AGI) 1.2.1 강인공지능의 실현 가능성에 관한 논쟁 2 역사 역사 하위섹션 토글하기 2.1 인공지능 이론의 발전 2.2 인공지능의 탄생(1943-1956) 2.2.1 인공두뇌학과 초기 신경 네트워크 2.2.2 튜링 테스트 2.2.3 게임 인공지능 2.2.4 상징 추론과 논리 이론 2.2.5 다트머스 컨퍼런스 1956년: AI의 탄생 2.3 황금기(1956~1974년) 2.3.1 작업들 2.3.1.1 탐색 추리 2.3.1.2 자연어 처리 2....

--- 키워드 (명사) 추출 중 ---
총 100개의 주요 키워드가 추출되었습니다.
추출된 키워드 (상위 20개): ['지능', '인공', '기술', '시스템', '연구', '문제', '기계', '컴퓨터', '인간', '사용', '프로그램', '공학', '대한', '분야', '사람', '문서', '다른', '이론', '대해', '스키']
해당 빈도 (상위 20개): [253, 186, 75, 70, 67, 66, 63, 57, 53, 51, 46, 40, 40, 35, 34, 30, 28, 27, 26, 25]


In [6]:
if keywords:
    print("\n--- 키워드 임베딩 진행 중 ---")
    word_embeddings = embed_words(keywords, model)

    if word_embeddings is not None:
        print("\n--- 3D 차원 축소 (TSNE) 진행 중 ---")
        reduced_word_embeddings_3d = reduce_dimensions_3d(word_embeddings)

        if reduced_word_embeddings_3d is not None:
            df_words = pd.DataFrame(reduced_word_embeddings_3d, columns=['Dimension 1', 'Dimension 2', 'Dimension 3'])
            df_words['Word'] = keywords
            df_words['Frequency'] = frequencies # 여기에 빈도(Frequency) 열 추가!
            print("\n3D 차원 축소 완료. 단어 데이터 프레임 준비 완료.")
        else:
            print("경고: 3D 차원 축소에 실패했습니다. 키워드 개수가 너무 적을 수 있습니다.")
    else:
        print("경고: 단어 임베딩에 실패했습니다.")
else:
    print("경고: 시각화할 키워드가 없습니다.")


--- 키워드 임베딩 진행 중 ---
100개의 단어를 임베딩 중...


Batches: 100%|██████████| 4/4 [00:00<00:00,  5.74it/s]



--- 3D 차원 축소 (TSNE) 진행 중 ---

3D 차원 축소 완료. 단어 데이터 프레임 준비 완료.


In [8]:
if 'df_words' in locals() and not df_words.empty:
    print("\n--- 3D 시각화 생성 중 (빈도 반영) ---")
    fig_3d = px.scatter_3d(df_words, 
                           x='Dimension 1', 
                           y='Dimension 2', 
                           z='Dimension 3',
                           color='Frequency', # 빈도에 따라 색상도 다르게 할 수 있습니다. (선택 사항)
                           size='Frequency',  # 점의 크기를 빈도에 비례하게 설정
                           hover_name='Word',
                           hover_data={'Frequency': True, 'Dimension 1': False, 'Dimension 2': False, 'Dimension 3': False}, # 호버 데이터에 빈도 포함
                           title=f"'{url_to_visualize}' 웹 페이지 주요 단어 임베딩 3D 시각화 (빈도 반영)",
                           height=700, width=1000)
    fig_3d.show()

    print("\n--- 시각화된 주요 키워드 목록 (빈도 포함) ---")
    # DataFrame에서 키워드와 빈도 출력 (더 정확)
    for index, row in df_words.iterrows():
        print(f"{index+1}. {row['Word']} (빈도: {int(row['Frequency'])})")
else:
    print("시각화할 단어 데이터가 준비되지 않았습니다. 이전 단계를 확인해주세요.")


--- 3D 시각화 생성 중 (빈도 반영) ---



--- 시각화된 주요 키워드 목록 (빈도 포함) ---
1. 지능 (빈도: 253)
2. 인공 (빈도: 186)
3. 기술 (빈도: 75)
4. 시스템 (빈도: 70)
5. 연구 (빈도: 67)
6. 문제 (빈도: 66)
7. 기계 (빈도: 63)
8. 컴퓨터 (빈도: 57)
9. 인간 (빈도: 53)
10. 사용 (빈도: 51)
11. 프로그램 (빈도: 46)
12. 공학 (빈도: 40)
13. 대한 (빈도: 40)
14. 분야 (빈도: 35)
15. 사람 (빈도: 34)
16. 문서 (빈도: 30)
17. 다른 (빈도: 28)
18. 이론 (빈도: 27)
19. 대해 (빈도: 26)
20. 스키 (빈도: 25)
21. 논리 (빈도: 25)
22. 전문가 (빈도: 25)
23. 해결 (빈도: 24)
24. 학습 (빈도: 23)
25. 주장 (빈도: 22)
26. 연구자 (빈도: 21)
27. 성공 (빈도: 21)
28. 개발 (빈도: 20)
29. 생각 (빈도: 20)
30. 프로젝트 (빈도: 20)
31. 인식 (빈도: 19)
32. 추론 (빈도: 19)
33. 개념 (빈도: 18)
34. 과학 (빈도: 18)
35. 언어 (빈도: 18)
36. 지식 (빈도: 17)
37. 신경망 (빈도: 17)
38. 모든 (빈도: 17)
39. 접근 (빈도: 17)
40. 가지 (빈도: 17)
41. 위해 (빈도: 16)
42. 튜링 (빈도: 16)
43. 형태 (빈도: 16)
44. 포함 (빈도: 16)
45. 목표 (빈도: 16)
46. 구가 (빈도: 15)
47. 또한 (빈도: 15)
48. 능력 (빈도: 15)
49. 매카시 (빈도: 15)
50. 정의 (빈도: 15)
51. 일반 (빈도: 15)
52. 처리 (빈도: 14)
53. 이용 (빈도: 14)
54. 주의 (빈도: 13)
55. 행동 (빈도: 13)
56. 방법 (빈도: 13)
57. 실제 (빈도: 13)
58. 우리 (빈도: 13)
59. 산업 (빈도: 13)
60. 세계 (빈도: 13)
61. 에